In [2]:
import pandas as pd
import csv

In [8]:
data_one = pd.read_csv("data/2000to2012.csv")
data_two = pd.read_csv("data/2012to2014.csv")
data_three = pd.read_csv("data/jan2015toDec2016.csv")
data_four = pd.read_csv("data/jan2017upwards.csv")

data_one.drop(columns=["block","street_name"])
data_two.drop(columns=["block","street_name"])
data_three.drop(columns=["block","street_name"])
data_four.drop(columns=["block","street_name"])

TypeError: unsupported operand type(s) for +: 'float' and 'str'